In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import sys
import csv

pd.set_option('display.max_columns', None)
rootdir = os.path.join(os.path.abspath(''), 'Assignment2_Company_ETL')

In [2]:
df_list = []
for root, dirs, files in tqdm(os.walk(rootdir)):
    for file in files:
        if file.endswith('.xlsx'):
            df = pd.read_excel(os.path.join(root, file), ignore_index=True)
            df_list.append(df.reindex(sorted(df.columns), axis=1))

54it [01:48,  2.00s/it]


In [3]:
data = pd.DataFrame(df_list)
data_mths = data[0:35]
data_yrs = data[35:38]
data_mths = data_mths.reset_index(drop=True)
data_yrs = data_yrs.reset_index(drop=True)
data = data.reset_index(drop=True)

In [4]:
# check rows and columns of each file
mths_dims = []
for i in range(len(data_mths[0])):
    mths_dims.append(data_mths[0][i].shape)
yrs_dims = []
for i in range(len(data_yrs[0])):
    yrs_dims.append(data_yrs[0][i].shape)

In [5]:
# check number of entries in total
print(sum([i[0] for i in mths_dims]))
print(sum([i[0] for i in yrs_dims]))

6211
6341


In [6]:
# check data types
data[0][0].dtypes

  Sailing DT                      datetime64[ns]
 Activity Feu                              int64
 Activity Teu                              int64
 Also Notify Name                        float64
 B/L No                                   object
 Booking Contact E-Mail                   object
 Booking Contact Phone Number            float64
 Booking No                               object
 CMDT                                     object
 CMDT Code                               float64
 Consignee                                object
 Consignee Address                        object
 Customs Description                      object
 DEL                                      object
 Export Name                              object
 FEU                                       int64
 Forward Name                             object
 Freight Term                             object
 MEA(CBM)                                float64
 Mty P/up CY                              object
 Notify Address     

In [7]:
# identify where differences in columns are coming from
dat1_cols = data[0][0].columns
for i in range(len(data[0])):
    set(data[0][i].columns) - set(data[0][0].columns)
    if (sum(data[0][i].columns != dat1_cols) > 0):
        a = set(data[0][i].columns) - set(data[0][0].columns)
        print('DF column differences with first DF {} \n'.format(str(a)))

DF column differences with first DF {' DEL EQ OFC', ' Forward Code ', ' ORG EQ OFC'} 

DF column differences with first DF {'  Trunk POD', ' Trunk POL', 'Export Name'} 

DF column differences with first DF {' DEL EQ OFC', ' Forward Code ', ' ORG EQ OFC'} 

DF column differences with first DF {'  Trunk POD', ' Trunk POL', 'Export Name'} 

DF column differences with first DF {'  Trunk POD', ' Trunk POL', 'Export Name'} 

DF column differences with first DF {'  Trunk POD', ' Trunk POL', 'Export Name'} 

DF column differences with first DF {'  Trunk POD', ' Trunk POL', 'Export Name'} 

DF column differences with first DF {'  Trunk POD', ' Trunk POL', 'Export Name'} 

DF column differences with first DF {'  Trunk POD', ' Trunk POL', 'Export Name'} 

DF column differences with first DF {'  Trunk POD', ' Trunk POL', 'Export Name'} 

DF column differences with first DF {'  Trunk POD', ' Trunk POL', 'Export Name'} 

DF column differences with first DF {'  Trunk POD', ' Trunk POL', 'Export Name'

In [8]:
# merging data
full_data = data[0][0]

for i in range(1, len(data[0])):
    full_data = full_data.append(data[0][i])

In [9]:
# cleaning column names
full_data.columns = full_data.columns.str.strip().str.lower().str.replace(" ", "_")
full_data.columns

Index(['sailing_dt', 'activity_feu', 'activity_teu', 'also_notify_name',
       'b/l_no', 'booking_contact_e-mail', 'booking_contact_phone_number',
       'booking_no', 'cmdt', 'cmdt_code', 'consignee', 'consignee_address',
       'customs_description', 'del', 'export_name', 'feu', 'forward_name',
       'freight_term', 'mea(cbm)', 'mty_p/up_cy', 'notify_address',
       'notify_name', 'pkg', 'pkg_unit', 'pod(booking)', 'pol(booking)', 'por',
       'rep_cmdt', 'rev.week', 'si_contact_e-mail', 'si_contact_phone_number',
       'shipper', 'shipper_address', 'teu', 'trunk_pod', 'trunk_pol',
       'trunk_vvd', 'vessel_name', 'wgt(kgs)', 'del_eq_ofc', 'forward_code',
       'org_eq_ofc', 'trunk_pod', 'trunk_pol', 'export_name'],
      dtype='object')

In [10]:
sum(full_data.duplicated())

2691

In [12]:
full_data['export_name']

,export_name,export_name
0,1103110007382 / CPH31107163 / MIJO\nFF SKAGEN A/S,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
801,NaN,NaN
802,NaN,NaN
803,NaN,NaN
804,NaN,NaN


In [26]:
col_unique_vals = []
for i in full_data.columns:
    if isinstance(full_data[i], pd.DataFrame):
        dt = 'dataframe'
        nu = sum(full_data[i].nunique())
    else :
        dt = full_data[i].dtypes
        nu = full_data[i].nunique()
    ds = full_data[i].shape
    col_unique_vals.append((i, nu))
    print("Column \'{}\' of type {} with shape {} has {} unique values.".format(i, dt, ds, nu))

Column 'sailing_dt' of type datetime64[ns] with shape (12552,) has 1505 unique values.
Column 'activity_feu' of type int64 with shape (12552,) has 74 unique values.
Column 'activity_teu' of type int64 with shape (12552,) has 66 unique values.
Column 'also_notify_name' of type object with shape (12552,) has 397 unique values.
Column 'b/l_no' of type object with shape (12552,) has 5313 unique values.
Column 'booking_contact_e-mail' of type object with shape (12552,) has 336 unique values.
Column 'booking_contact_phone_number' of type object with shape (12552,) has 351 unique values.
Column 'booking_no' of type object with shape (12552,) has 4891 unique values.
Column 'cmdt' of type object with shape (12552,) has 478 unique values.
Column 'cmdt_code' of type float64 with shape (12552,) has 481 unique values.
Column 'consignee' of type object with shape (12552,) has 1736 unique values.
Column 'consignee_address' of type object with shape (12552,) has 2120 unique values.
Column 'customs_des

In [34]:
max_nu = max([i[1] for i in col_unique_vals])
max_nu

5313